# Chapter 13; 파이썬 모델링 라이브러리

- 13.1 pandas와 모델 코드의 인터페이스

In [2]:
# 모델 개발의 일반적인 흐름은 데이터를 불러오고 정제하는 과정은 pandas를 그 후 모델 개발을 위해 모델링 라이브러리로 넘어가게 된다.
# 모델을 개발하는 과정에서 중요한 단계는 특징을 선택하고 추출하는 피처 엔지니어링인데 원시 데이터셋으로부터 모델링에서 유용할 수 있는 정보를 추출하는 변환이나 분석과정을 일컫는다.
# 이 책에서 살펴본 데이터 요약이나 GroupBy 도구들이 피처 엔지니어링 과정에서 자주 사용된다.

In [3]:
# "좋은" 피처 엔지니어링에 대한 자세한 내용은 이 책의 범위를 벗어나므로 pandas를 이용한 데이터 조작과 모델링 사이를 편리하게 오갈 수 있는 방법을 설명하겠다.

In [4]:
# pandas와 다른 분석 라이브러리는 주로 NumPy 배열을 사용해서 연계할 수 있다.
# DataFrame을 NumPy 배열로 변환하려면 .values 속성을 이용한다.

In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    "y": [-1.5, 0., 3.6, 1.3, -2.]})

In [8]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [9]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [10]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [11]:
# 다시 DataFrame으로 되돌리려면 앞서 공부했던 것처럼 2차원 ndarrary와 필요하다면 컬럼 이름 리스트를 넘겨서 생성할 수 있다.

In [12]:
df2 = pd.DataFrame(data.values, columns=["one", "two", "three"])

In [13]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


- NOTE_ .values 속성은 데이터가 한 가지 타입(예를 들면 모두 숫자형)으로 이루어져 있다는 가정 하에 사용된다.
- 만약 데이터 속성이 한 가지가 아니라면 파이썬 객체의 ndarrary가 반환될 것이다.

In [14]:
df3 = data.copy()

In [15]:
df3["strings"] = ["a", "b", "c", "d", "e"]

In [16]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [17]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [18]:
# 어떤 모델은 전체 컬럼 중 일부만 사용하고 싶은 경우도 있을 것이다. 이 경우 loc을 이용해서 values 속성에 접근하기 바란다.

In [19]:
model_cols = ["x0", "x1"]

In [20]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [21]:
# 어떤 라이브러리는 pandas를 직접 지원하기도 하는데 위에서 설명한 과정을 자동으로 처리해준다.
# DataFrame에서 NumPy 배열로 변환하고 모델 인자의 이름을 출력 테이블이나 Series의 컬럼으로 추가한다.
# 아니라면 이런 메타 데이터 관리를 수동으로 직접 해야한다.

In [22]:
# 12장에서 pandas의 Categorical형과 pandas.get_dummies 함수를 살펴봤다.
# 에제 데이터셋에 숫자가 아닌 컬럼이 있다고 가정하자.

In [23]:
data["category"] = pd.Categorical(["a", "b", "a", "a", "b"],
                                  categories=["a", "b"])

In [24]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [25]:
# 만일 "category" 컬럼을 더미 값으로 치환하고 싶다면 더미값을 생성하고 "category" 컬럼을 삭제한 다음 결과와 합쳐야 한다.

In [26]:
dummies = pd.get_dummies(data.category, prefix="category")

In [27]:
data_with_dummies = data.drop("category", axis=1).join(dummies)

In [28]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


In [29]:
# 특정 통계 모델을 더미값으로 피팅하는 기법도 있다. 단순히 숫자형 컬럼만 가지고 있는 게 아니라면 다음 절에서 살펴볼 Patsy를 사용하는 편이 더 단순하다.
# 또한 에러를 일으킬 가능성도 줄여준다.